# Analysis of Movie Industry


insert pretty picture here, sponsored by microsoft, microsoft logo

## Overview

In light of Microsoft's desire to expand their enterprise into the movie sector, we have analyzed various qualities of movie data to help develop some insights to help maximize Microsoft's ROI for their potential startup. 

{PUT SUMMARY OF INSIGHTS HERE}

Since we are both relatively unfamiliar with the business end of the industry, supplementary research on the working of the industry as a whole were considered to help drive and focus our analysis. Access sources HERE.

Data was analyzed from IMDB, Box Office Mojo, and The-Numbers.com . (include hyperlinks)
From each of the above 

## Business Problem

picture?

Highlight specific areas of focus
- start up company
- changing industry
- good strategies {use movie terms: release strategy, dump month, etc}

## Unpacking the Data

*Recognize credentials of sources listed above as credible, good data supply, etc



In [5]:
import pandas as pd

We adjusted each of the monetary values to accomodate for inflation. [Here](https://github.com/datadesk/cpi#:~:text=Working%20with%20Python,the%20Bureau%20of%20Labor%20Statistics) is the link to the package info.

In [ ]:
# if you do not have this cpi package, you can download as:
# pip install cpi (!pip install cpi if you'd like in your Jupyter notebook)

In [7]:
import cpi
cpi.update() # run to make sure current Consumer Price Index is being used in calculations

In [8]:
########## JAMIE CLEANING CODE ###########

In [12]:
bomMovieGross = pd.read_csv('data/bom.movie_gross.csv.gz')

imdbNameBasics = pd.read_csv('data/imdb.name.basics.csv.gz')

imdbTitleAkas = pd.read_csv('data/imdb.title.akas.csv.gz')

imdbTitleBasics = pd.read_csv('data/imdb.title.basics.csv.gz')

imdbTitleCrew = pd.read_csv('data/imdb.title.crew.csv.gz')

imdbTitlePrincipals = pd.read_csv('data/imdb.title.principals.csv.gz')

imdbTitleRatings = pd.read_csv('data/imdb.title.ratings.csv.gz')

rtMovieInfo = pd.read_csv('data/rt.reviews.tsv.gz', sep='\t', encoding='windows-1252')

rtReviews = pd.read_csv('data/rt.reviews.tsv.gz', sep='\t', encoding='windows-1252')

tmdbMovies = pd.read_csv('data/tmdb.movies.csv.gz')

tnMovieBudgets = pd.read_csv('data/tn.movie_budgets.csv.gz')

#### kaggle data csv read here ####

### Cleaning Movie Budget data from [The-Numbers.com](https://www.the-numbers.com/)

In [13]:
tnMovieBudgets.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


The production budget, domestic gross, and worldwide gross columns need to be converted into numbers:

In [ ]:
money_cols = ['worldwide_gross', 'domestic_gross', 'production_budget']
tnMovieBudgets[money_cols] = tnMovieBudgets[money_cols].applymap(lambda x: int(x.replace('$','').replace(',','')))

In [25]:
tnMovieBudgets[money_cols].head(1)

,worldwide_gross,domestic_gross,production_budget
0,2776345279,760507625,425000000


Then, we adjusted the dates to create separate columns for month and year released:

In [23]:
tnMovieBudgets['release_date'] = pd.to_datetime(tnMovieBudgets['release_date'], errors='coerce') # what does coerce do?
tnMovieBudgets['release_date']

tnMovieBudgets['month_released'] = tnMovieBudgets['release_date'].apply(lambda x: str(x))
tnMovieBudgets['month_released'] = tnMovieBudgets['month_released'].apply(lambda x: int(x[5:7]))

tnMovieBudgets['year'] = tnMovieBudgets['release_date'].apply(lambda x: str(x))
tnMovieBudgets['year'] = tnMovieBudgets['year'].apply(lambda x: int(x[:4]))

tnMovieBudgets.drop(labels='release_date', axis=1, inplace=True)

In [26]:
tnMovieBudgets[['month_released', 'year']].head(1)

,month_released,year
0,12,2009


### Cleaning [Box Office Mojo](https://www.boxofficemojo.com/) data

In [27]:
bomMovieGross.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [ ]:
bomMovieGross['foreign_gross'] = bomMovieGross['foreign_gross'].apply(lambda x: float(str(x).replace(',','')))

### Adjusting tnMovieBudgets and bomMovieGross for inflation

First, we made sure all NaN values were an integer:

In [ ]:
bomMovieGross['foreign_gross'].fillna(value=0, inplace=True)



Then, we converted the numbers using **cpi.inflate**:

In [293]:

bomMovieGross['domestic_adj'] = bomMovieGross.apply(lambda x: cpi.inflate(x.domestic_gross, x.year), axis=1)

bomMovieGross['foreign_adj'] = bomMovieGross.apply(lambda x: cpi.inflate(x.foreign_gross, x.year), axis=1)

In [294]:
bomMovieGross.rename(mapper={'title' : 'primary_title'}, axis=1, inplace=True)

In [295]:
bomMovieGross.head()

,primary_title,studio,domestic_gross,foreign_gross,year,domestic_adj,foreign_adj
0,Toy Story 3,BV,415000000.0,652000000.0,2010,4.925641e+08,7.738598e+08
1,Alice in Wonderland (2010),BV,334200000.0,691300000.0,2010,3.966625e+08,8.205050e+08
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000.0,2010,3.513229e+08,7.884587e+08
3,Inception,WB,292600000.0,535700000.0,2010,3.472874e+08,6.358231e+08
4,Shrek Forever After,P/DW,238700000.0,513900000.0,2010,2.833134e+08,6.099487e+08


After some cursory analysis, we realized that we wanted to measure averages of domestic and worldwide gross **combined**, so we created a new column, 'total_gross':

In [ ]:

tnMovieBudgets['total_gross'] = tnMovieBudgets['worldwide_adj'] + tnMovieBudgets['domestic_adj']

In [ ]:
########### MOOMEN CLEANING CODE #############

### Relevant Variables

Explain each data set

In [3]:
# show head of dataset 1

In [ ]:
# show head of dataset 2

In [ ]:
# show head of dataset 3

In [ ]:
# show head of dataset 4

In [ ]:
# show head of dataset 5

In [ ]:
# show head of dataset 6

## Cleaning / Reorganizing Data

Summarize cleaning process

In [ ]:
################ JAMIE CLEANING #################

In [ ]:
################ CHRISTOS CLEANING ################

### Sub-elements of cleaning (such as important functions)

picture to divide the two sections

## Analysis

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

import code.visualizations as viz

%matplotlib inline

In [ ]:
#################### JAMIE VISUALIZATIONS ####################


### Analysis of Variable #1

desc here

### Analysis of Variable #2

### Analysis of Variable #3

### Analysis of Variable #4

In [ ]:
################ CHRISTOS VISUALIZATIONS ####################


### Analysis of Variable #1

desc here

### Analysis of Variable #2

### Analysis of Variable #3

### Analysis of Variable #4

# Conclusions

summaries in nice bullet points 

# Next Steps

next steps